In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.append('..')

In [2]:
from metric import compute_metric
import pickle
import pandas as pd
import matplotlib.pyplot as plt


runtime = '2023-09-26_23-20-59'
thres = 0.5
plt.figure()
idx = 4
data_dir = f'../datasets/Case-study/binary2.tsv'
name = f'model{idx}'
pkl_path = f'../result/binary/{runtime}/{name}'
results = sorted(os.listdir(pkl_path))
num_split = len(results)
dataset = pd.read_csv(data_dir, sep='\t')
dataset['seqlen'] = dataset['text'].apply(lambda x: len(x))
for i in range(num_split):
    pkl_dir = f'{pkl_path}/{results[i]}'
    with open(pkl_dir, 'rb') as fr:
        pkl = pickle.load(fr)
        outputs, labels = pkl
        compute_metric(labels[:, 1], outputs[:, 1], thres=thres)
    dataset[i] = outputs[:, 1]
column_names = range(num_split)
averages = dataset[column_names].mean(axis=1)
print('Mean')
auroc_val, fpr, tpr = compute_metric(labels[:, 1], averages, thres=thres)
dataset['avg'] = averages

0.8899	1.0	nan	0.0	nan
0.8991	1.0	nan	0.0	nan
0.8991	1.0	nan	0.0	nan
0.9266	1.0	nan	0.0	nan
0.945	1.0	nan	0.0	nan
Mean
0.9266	1.0	nan	0.0	nan


/data/s853wang/ACP/DUO-ACP/notebook/../metric.py:26: RuntimeWarning: invalid value encountered in scalar divide
  specificity = tn / (tn + fp)
/data/s853wang/miniconda3/envs/ACP/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:1123: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn(
/data/s853wang/ACP/DUO-ACP/notebook/../metric.py:26: RuntimeWarning: invalid value encountered in scalar divide
  specificity = tn / (tn + fp)
/data/s853wang/miniconda3/envs/ACP/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:1123: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn(
/data/s853wang/ACP/DUO-ACP/notebook/../metric.py:26: RuntimeWarning: invalid value encountered in scalar divide
  specificity = tn / (tn + fp)
/data/s853wang/miniconda3/envs/ACP/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:1123: UndefinedMetricWarning: No negative samples in y_t

<Figure size 640x480 with 0 Axes>

In [3]:
from metric import compute_metric_labelwise, compute_metric_score
import numpy as np
import pickle

folder = f'../result/multiclass/{runtime}'
labels, outputs = [], []
for k in range(1, 11):
    result_file = f'{folder}/model3/{k}.pkl'
    res = pickle.load(open(result_file, 'rb'))
    output, label = res
    labels.append(label)
    outputs.append(output)

labels = np.mean(labels, axis=0)
outputs = np.mean(outputs, axis=0)
print(outputs.shape)

#outputs
compute_metric_labelwise(labels, outputs)
compute_metric_score(labels, outputs)

(109, 7)
0.933	0.835	0.803	0.667	0.871	0.664
0.902	0.771	0.887	0.86	0.79	0.56
0.96	0.872	0.772	0.797	0.863	0.764
0.96	0.89	0.846	0.914	0.846	0.76
0.957	0.881	0.905	0.935	0.854	0.757
0.981	0.908	0.8	0.91	0.848	0.786
0.927	0.789	0.207	0.777	0.343	0.401
Average
0.946	0.849	0.746	0.837	0.774	0.67
------Macro------
Macro accuracy 0.8492791612057667
Macro precision 0.745595165568407
Macro recall 0.8858448972574744
Macro f1-score 0.7736454368028346
------Micro------
Micro accuracy 0.8492791612057667
Micro precision 0.7764350453172205
Micro recall 0.8624161073825504
Micro f1-score 0.8171701112877583
------Hamming loss------
Hamming loss 0.15072083879423329


0.8492791612057667

In [4]:
from pyteomics import fasta
from util import all_labels
fa = fasta.read('../datasets/Case-study/multiclass2.fasta')
seq_list = []
for x in fa:
    seq_list.append(x.sequence)
    
df = pd.DataFrame(outputs, columns=all_labels)
df['seq'] = seq_list
df['binary'] = averages
df = df.sort_values('binary', ascending=False)
df.to_csv('out.csv', index=False)